# Sensor placement for quality monitoring

### Initialize EPANET Python Toolkit (EPyT)

You should always begin with this command to import the toolkit.

[EPyT](https://github.com/OpenWaterAnalytics/EPyT) is available on [PyPI](https://pypi.org/project/epyt/) and can be installed via `pip install epyt`. To upgrade to the latest version if it's already installed, use `pip install --upgrade epyt`.

In [ ]:
%pip install epyt pymoo numpy==1.23

In [ ]:
from epyt import epanet
import numpy as np
from math import comb

### Introduction

Large-scale networks are difficult to monitor, and it is not possible to have sensors everywhere in the system, mainly due to costs. For this reason, we need to select the best locations to install sensors, in order to guarantee certain criteria, such as:  
   - Network coverage  
   - Detection delay  
   - Impact (e.g., number of people affected)   
     
                                                      
Sensor placement requires the following steps:
   1. Create simulation scenarios of contamination events
   2. Execute the simulation scenarios
   3. Measure the impact
   4. Formulate and solve an optimization problem
   5. If multiple objectives, costruct the Pareto front and select one solution

### Load EPANET Network and MSX

In [ ]:
G = epanet('data/BWSN_Network_1.inp')    # Load EPANET Input file

### Simulation Setup

In [ ]:
t_d = 5 # days
G.printv(t_d)

G.setTimeSimulationDuration(t_d*24*60*60)

### Get Network data

In [ ]:
demand_pattern = G.getPattern()
roughness_coeff = G.getLinkRoughnessCoeff()
node_id = G.getNodeNameID()
Nnodes = G.getNodeCount()
# *Setup uncertainties*

### Scenarios

In [ ]:
Ns = 50 # Number of scenarios to simulate
u_p = 0.2   # pattern uncertainty
u_r = 0.2   # roughness coefficient uncertainty
G.printv(u_p)
G.printv(u_r)

##### Create the scenarios - first without contamination, to compute the bounds

In [ ]:
max_inj_conc = 10.0
inj_start_time = 1 
inj_duration = 24   # maximum duration of 12 hours
G.printv(max_inj_conc)
G.printv(inj_start_time)
G.printv(inj_duration)

In [ ]:
# Generate injection scenarios
inj_sc = np.column_stack((
    np.random.randint(1, Nnodes + 1, size=Ns),  # Random integers between 1 and Nnodes inclusive
    max_inj_conc * np.ones(Ns),
    inj_start_time * np.ones(Ns),
    inj_duration * np.ones(Ns)
))
G.printv(inj_sc)

In [ ]:
# *Create the coverage matrix*
# The coverage matrix is a binary matrix, where earh row is a scenario, and 
# each column is a node. If a certain contamination scenario affects (can reach) 
# a certain node, the flag is true, else false.

K = np.zeros((Ns, Nnodes))
for i in range(Ns):
    print(f'Iteration {i+1}')
    
    # Randomize demands
    r_p = -u_p + 2*u_p*np.random.rand(demand_pattern.shape[0], demand_pattern.shape[1])
    new_demand_pattern = demand_pattern + demand_pattern*r_p
    G.setPatternMatrix(new_demand_pattern) # Set new patterns
    
    # Randomize pipe roughness
    r_r = -u_r + 2*u_r*np.random.rand(len(roughness_coeff))
    new_roughness_coeff = roughness_coeff + roughness_coeff*r_r
    G.setLinkRoughnessCoeff(new_roughness_coeff) # Set new roughness coefficients
    
    c = np.zeros(Nnodes)
    c[int(inj_sc[i, 0])-1] = inj_sc[i, 1]  
    G.setNodeInitialQuality(c.tolist())
    
    qsp = G.getComputedTimeSeries().NodeQuality
    
    K[i, :] = np.max(qsp, axis=0) > 0

### Optimization problem
  - Avoid having sensors to monitor the same area
  - Some scenarios affect all nodes, and some others, may not affect any node.    

This means: select those columns that have ones, so that there is at least one positive flag '1' in as many rows as possible  

  - $Y   $  is the vector of "scenario observability" (number of scenarios × 1)
  - $u $  is the solution, a binary vector (number of nodes × 1)
  - $K $  is the coverage matrix (number of scenarios × number of nodes) <br>

    $ Y = Ku $ 

The optimization problem is formed (in its simplest form) as  <br>
$$ arg max f(u) $$   $$ u \epsilon \{0, 1\}^N $$
 $$ Σ u_i = m $$    
                                                  

Where $f(u)$  is the objective function. The simplest function is "the percentage of scenarios which can be 'detected' by the sensors".
We can have multiple functions, and in that case, we need multi-objective optimization. Solving for a single solution

In [ ]:
# random selection of nodes
np.random.seed(1)
Nsensors = 5    # number of sensors
G.printv(Nsensors)

In [ ]:

total_combinations = comb(Nnodes, Nsensors) # 275 Millions for 5 sensors in 129 nodes
G.printv(total_combinations)

In [ ]:
# Create a random solution
s = np.random.permutation(Nnodes)[:Nsensors] # return random
G.printv(s)

In [ ]:
# solution
u = np.zeros((Nnodes, 1))
u[s-1] = 1
# compute output
Y = np.dot(K, u)
G.printv(Y.T)

In [ ]:
# How many non-zero elements in Y?
perc_coverage = np.sum(Y > 0) / Nsensors * 100
G.printv(perc_coverage)

In [ ]:
# *Larger scenario dataset*

  - How many scenarios should we create?
  - In general, the more scenarios, the more robust your solution
  - Simulate contamination events at different time steps
  - Simulate different hydraulic conditions   
  
#### Load results from BWSN 2006 competition (37K scenarios)

In [ ]:
import scipy.io

# Load Kall.mat
data = scipy.io.loadmat('data/Kall.mat')
Kall = data['Kall']

# Get the size of Kall
Nsall, _ = Kall.shape
G.printv(Nsall)

#### Repeat solution computation for the larger dataset

In [ ]:
Nsensors = 5    # number of sensors
G.printv(Nsensors)

In [ ]:
# Create a random solution
s = np.random.permutation(Nnodes)[:Nsensors]
G.printv(s)

In [ ]:
# Solution
u = np.zeros((Nnodes, 1))
u[s - 1] = 1 
# Compute output
Y = np.dot(Kall, u)
# How many non-zero elements in Y?
perc_coverage = np.sum(Y > 0) / Nsall * 100
G.printv(perc_coverage)

In [ ]:
# *Randomized solutions for multiple sensors*
# How many sensors to install? Let's create a lot of random solutions
 
epochs = 1000
maxsensors = 20
G.printv(epochs)
G.printv(maxsensors)

In [ ]:
fx = np.zeros((epochs, 2))

for i in range(epochs):
    print(i)
    ns = np.random.randint(1, maxsensors + 1)  
    s = np.random.permutation(Nnodes)[:ns]  
    u = np.zeros((Nnodes, 1))
    u[s - 1] = 1  
    Y = np.dot(Kall, u)
    fx[i, :] = [ns, np.sum(Y > 0) / Nsall * 100]
    
print('fx =\n', fx.shape)

Plotting the results

In [ ]:
import matplotlib.pyplot as plt

# Plot individual points
plt.plot(fx[:, 0], fx[:, 1], 'x', markersize=3)

# Plot settings
plt.grid(True)
plt.xlabel('Number of Sensors', fontsize=8)
plt.ylabel('Percentage Coverage', fontsize=8)
plt.title('Percentage Coverage vs Number of Sensors', fontsize=10)

plt.ylim(0, 90)
plt.yticks(range(0, 90, 20))

plt.xlim(0, 20)
plt.xticks(range(0, 21, 4))

# Find maximum coverage for each number of sensors
maxfx = np.zeros((maxsensors, 2))
for i in range(1, maxsensors + 1):
    maxfx[i - 1] = [i, np.max(fx[fx[:, 0] == i, 1])]

# Plot maximum coverage
plt.plot(maxfx[:, 0], maxfx[:, 1], 'r-')

plt.show()


<b>Question:</b> Is this a valid way of solving problems
### Solving the optimization problem
  - Integer/Mixed Integer Quadradic Programming
  - Evolutionary Optimization (Single-objective)
  - Evolutionary Optimization (Multi-objective) Genetic algorithms

In [ ]:
## TESTTTTTTTTTTTTTTTTTTTTTTTTTTT

np.random.seed(2)
Nsensors = 5
print('Nsensors =', Nsensors)

In [ ]:
Nsall, Nnodes = Kall.shape
print('Nsall =', Nsall)
print('Nnodes =', Nnodes)

In [ ]:
def gafunc(x):
    return calc_score(x, Kall, Nsall)

In [ ]:
def calc_score(x, Kall, Nsall):
    # Calculate the score
    scores = np.max(Kall[:, np.round(x).astype(int)], axis=1)
    score = -np.sum(scores) / Nsall * 100
    return score


In [ ]:
def calc_score_multi(x, Kall, Nsall, Tall):
    score1 = -np.sum(np.max(Kall[:, np.round(x).astype(int)], axis=1)) / Nsall * 100
    score2 = np.median(np.min(Tall[:, np.round(x).astype(int)], axis=1))
    return [score1, score2]